In [12]:
import pandas as pd

def SMA(array, n):
    
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()

def RSI(array, n):
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(span=n).mean() / loss.abs().ewm(span=n).mean()  # FIXED
    return 100 - 100 / (1 + rs)

### Buy the position when:

 * weekly RSI(30)  ≥  daily RSI(30)  >  70
 * Close  >  MA(10)  >  MA(20)  >  MA(50)  >  MA(100)

### Close the position when:
 * Daily close is more than 2% below MA(10)
 * 8% fixed stop loss is hit

In [24]:
class System(Strategy):
    d_rsi = 30  
    w_rsi = 30  
    level = 70  
    
    def init(self):
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        self.weekly_rsi = resample_apply('W-FRI', RSI, self.data.Close, self.w_rsi)
        
    def next(self):
        price = self.data.Close[-1]

        if (not self.position and
            self.daily_rsi[-1] > 70 and
            self.weekly_rsi[-1] > 70 and
            self.ma10[-1] > self.ma20[-1] and
            price > self.ma10[-1]):

            print("BUY Signal Triggered!")
            self.buy(sl=.92 * price)

        elif price < 0.98 * self.ma10[-1]:
            print("SELL Signal Triggered!")
            self.position.close()


In [26]:
import yfinance as yf
from backtesting import Backtest

# Download stock data
GOOGLE = yf.download("GOOG", start="2018-01-01", end="2023-06-30", auto_adjust=True)

# Drop MultiIndex if necessary
if isinstance(GOOGLE.columns, pd.MultiIndex):
    GOOGLE.columns = GOOGLE.columns.droplevel(1)  

if GOOGLE.columns[0] == "Close":
    GOOGLE = GOOGLE[['Open', 'High', 'Low', 'Close', 'Volume']]  

# Run backtest

backtest = Backtest(GOOGLE, System, commission=0.002)
stats = backtest.run()

# Print results
print("Backtest completed!")
stats


[*********************100%***********************]  1 of 1 completed

SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal Triggered!
SELL Signal

Start                     2018-01-02 00:00:00
End                       2023-06-29 00:00:00
Duration                   2004 days 00:00:00
Exposure Time [%]                     7.59768
Equity Final [$]                  10634.09687
Equity Peak [$]                   11088.85957
Commissions [$]                     159.34544
Return [%]                            6.34097
Buy & Hold Return [%]               122.39725
Return (Ann.) [%]                     1.12737
Volatility (Ann.) [%]                 5.05455
CAGR [%]                               0.7761
Sharpe Ratio                          0.22304
Sortino Ratio                         0.32337
Calmar Ratio                          0.16022
Max. Drawdown [%]                    -7.03653
Avg. Drawdown [%]                    -3.07236
Max. Drawdown Duration      666 days 00:00:00
Avg. Drawdown Duration      179 days 00:00:00
# Trades                                    4
Win Rate [%]                             50.0
Best Trade [%]                    

In [ ]:
%%time

backtest.optimize(d_rsi=range(10, 35, 5),
                  w_rsi=range(10, 35, 5),
                  level=range(30, 80, 10))

In [6]:
backtest.plot()

GridPlot(id='p1223', ...)

### Result:
##### While the strategy doesn't perform as well as simple buy & hold, it does so with significantly lower exposure (time in market).
##### In conclusion, to test strategies on multiple time frames, you need to pass in OHLC data in the lowest time frame, then resample it to higher time frames, apply the indicators, then resample back to the lower time frame, filling in the in-betweens.

In [7]:
#These are the trades that would executed with this strategy.
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,35,200,207,282.95478,275.00,-278.41730,-0.028113,2005-06-06,2005-06-15,9 days
1,32,215,235,299.49780,295.01,-143.60960,-0.014984,2005-06-27,2005-07-26,29 days
2,25,304,328,372.60372,408.70,902.40700,0.096876,2005-11-01,2005-12-06,35 days
3,22,352,357,472.21254,451.17,-462.93588,-0.044562,2006-01-11,2006-01-19,8 days
